In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.nn.modules.distance import PairwiseDistance
from datasets.AIHubDataset import AIHubDataset
from validate_aihub import validate_aihub
from tqdm import tqdm
from model.inceptionresnetv2 import InceptionResnetV2Triplet
from model.mobilenetv2 import MobileNetV2Triplet
from model.resnet import (
    Resnet18Triplet,
    Resnet34Triplet,
    Resnet50Triplet,
    Resnet101Triplet,
    Resnet152Triplet,
)

In [22]:
def set_model_architecture(model_architecture, pretrained, embedding_dimension):
    if model_architecture == "resnet18":
        model = Resnet18Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    elif model_architecture == "resnet34":
        model = Resnet34Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    elif model_architecture == "resnet50":
        model = Resnet50Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    elif model_architecture == "resnet101":
        model = Resnet101Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    elif model_architecture == "resnet152":
        model = Resnet152Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    elif model_architecture == "inceptionresnetv2":
        model = InceptionResnetV2Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    elif model_architecture == "mobilenetv2":
        model = MobileNetV2Triplet(
            embedding_dimension=embedding_dimension, pretrained=pretrained
        )
    print("Using {} model architecture.".format(model_architecture))

    return model

In [23]:
def set_model_gpu_mode(model):
    flag_train_gpu = torch.cuda.is_available()
    flag_train_multi_gpu = False

    if flag_train_gpu and torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        model.cuda()
        flag_train_multi_gpu = True
        print("Using multi-gpu training.")

    elif flag_train_gpu and torch.cuda.device_count() == 1:
        model.cuda()
        print("Using single-gpu training.")

    return model, flag_train_multi_gpu

In [24]:
def set_optimizer(optimizer, model, learning_rate):
    if optimizer == "sgd":
        optimizer_model = optim.SGD(
            params=model.parameters(),
            lr=learning_rate,
            momentum=0.9,
            dampening=0,
            nesterov=False,
            weight_decay=1e-5,
        )

    elif optimizer == "adagrad":
        optimizer_model = optim.Adagrad(
            params=model.parameters(),
            lr=learning_rate,
            lr_decay=0,
            initial_accumulator_value=0.1,
            eps=1e-10,
            weight_decay=1e-5,
        )

    elif optimizer == "rmsprop":
        optimizer_model = optim.RMSprop(
            params=model.parameters(),
            lr=learning_rate,
            alpha=0.99,
            eps=1e-08,
            momentum=0,
            centered=False,
            weight_decay=1e-5,
        )

    elif optimizer == "adam":
        optimizer_model = optim.Adam(
            params=model.parameters(),
            lr=learning_rate,
            betas=(0.9, 0.999),
            eps=1e-08,
            amsgrad=False,
            weight_decay=1e-5,
        )

    return optimizer_model

In [25]:
# dataroot = "data/face-image/train_aihub_family"
# lfw_dataroot = "data/face-image/lfw_224"
aihub_dataroot = "data/face-image/test_aihub_family"
# training_dataset_csv_path = "aihub_train.csv"
epochs = 2  # 150
# iterations_per_epoch = 10  # 5000
model_architecture = "resnet34"
pretrained = True  # False
embedding_dimension = 512
# num_human_identities_per_batch = 32
# batch_size = 100  # 544
lfw_batch_size = 200
# resume_path = ""
num_workers = 4
optimizer = "adagrad"
learning_rate = 0.075
margin = 0.2
image_size = 140
use_semihard_negatives = False
training_triplets_path = None

In [26]:
flag_training_triplets_path = False
start_epoch = 0

if training_triplets_path is not None:
    flag_training_triplets_path = (
        True  # Load triplets file for the first training epoch
    )

In [27]:
# Define image data pre-processing transforms
aihub_mean = [0.5444, 0.4335, 0.3800]
aihub_std = [0.2672, 0.2295, 0.2156]
aihub_transforms = transforms.Compose(
    [
        transforms.Resize(size=(image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

In [28]:
TASK_CATEGORY = "Family"
aihub_dataloader = torch.utils.data.DataLoader(
    
    dataset=AIHubDataset(
        dir=aihub_dataroot,
        pairs_path=f"data/pairs/test/pairs_{TASK_CATEGORY}.txt",
        transform=aihub_transforms,
    ),
    batch_size=lfw_batch_size,
    num_workers=num_workers,
    shuffle=False,
)

In [29]:
# Instantiate model
model = set_model_architecture(
    model_architecture=model_architecture,
    pretrained=pretrained,
    embedding_dimension=embedding_dimension,
)

# Load model to GPU or multiple GPUs if available
model, flag_train_multi_gpu = set_model_gpu_mode(model)

# Set optimizer
optimizer_model = set_optimizer(
    optimizer=optimizer, model=model, learning_rate=learning_rate
)

Using resnet34 model architecture.
Using single-gpu training.


In [30]:
# Evaluation pass on LFW dataset
print(f"Task category, Pre-trained")
print(f"{TASK_CATEGORY}, {pretrained}")
for epoch in range(epochs):
    best_distances = validate_aihub(
        model=model,
        aihub_dataloader=aihub_dataloader,
        model_architecture=model_architecture,
        epoch=epoch
    )
    break

Task category, Pre-trained
Family, True
Validating on AIHUB! ...


100%|██████████| 30/30 [00:16<00:00,  1.77it/s]


Accuracy on AIHUB: 0.8465+-0.0098	Precision 0.8475+-0.0252	Recall 0.8470+-0.0242	ROC Area Under Curve: 0.9300	Best distance threshold: 0.53+-0.01	TAR: 0.1067+-0.0176 @ FAR: 0.0010
Accuracy on AIHUB, Precision, Recall, ROC Area Under Curve, Best distance threshold TAR, FAR
0.8465+-0.0098, 0.8475+-0.0252, 0.8470+-0.0242,0.9300, 0.53+-0.01,0.1067+-0.0176, 0.0010
